In [385]:
from bs4 import BeautifulSoup
import requests
import random
import re, time

In [62]:
USER_AGENT_LIST = [
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
    "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
    "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
    "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

In [388]:
def get_response(url,params):
    headers = {'user-agent':random.choice(USER_AGENT_LIST)}

    resp = requests.get(url,
                        headers=headers,
                        params=params,
                        timeout=3)
    resp.encoding = "UTF-8"
    resp.raise_for_status()
    soup = BeautifulSoup(resp.content, 'lxml')
    return soup

def scholar_responce(url, query, start_num, start_y, end_y):
    params = {'start':start_num,
              'q':query,
              'hl':'en',
              'as_sdt':'0,9',
              'as_ylo':end_y,
              'as_yhi':start_y}
    resp = get_response(url, params)
    scholar_unit = resp.select('div.gs_r.gs_or.gs_scl')
    np_check = True if resp.select_one('td[align=left] > a') else False
    return scholar_unit, np_check


def scholar_kitchen(soup_unit):
    Ad = soup_unit.select_one('div.gs_a').text.replace(u'\xa0', u' ')
    Tails = soup_unit.select('div.gs_fl > a')
    TailsT = [x.text for x in Tails]
    KP_CD = [i for i,j in enumerate(TailsT) if re.match('^Cited*',j) is not None]
    KP_url = soup_unit.select_one('h3.gs_rt > a[href]')
    KP_pdfurl = soup_unit.select_one('div.gs_or_ggsm > a[href]')
    KP_desc = soup_unit.select_one('div.gs_rs')
    
    Title = soup_unit.select_one('h3.gs_rt').text
    FAuthor = re.split(r',| - ',Ad)[0]
    Descript = KP_desc.text.replace(u'\xa0', u' ') if KP_desc is not None else ''
    Cite_count =  Tails[KP_CD[0]].text.split('Cited by ')[1] if len(KP_CD) is not 0 else ''
    Url = KP_url.attrs['href'] if KP_url is not None else ''
    PDF_url = KP_pdfurl.attrs['href'] if KP_pdfurl is not None else ''
    GS_ID = soup_unit.attrs['data-cid']
    
    Result = {'Title':Title,
             'FAuthor':FAuthor,
             'Url':Url,
             'PDFurl':PDF_url,
             'Description':Descript,
             'Cite_count':Cite_count,
             'GS_ID':GS_ID}
    return Result

In [389]:
def scholar_crawler(query, start_y, end_y):
    gscholar='https://scholar.google.com.tw/scholar'
    NP_state=True
    ingreds = []
    start_num = 0
    print('Start working on Crawling Job: 「Paper related to '+ 
          query +' from '+ str(start_y) +' to '+ str(end_y) + '」')
    while NP_state:
        soups, np_check = scholar_responce(gscholar, query, start_num, start_y, end_y)
        for i, soup in enumerate(soups):
            try:
                ingreds.append(scholar_kitchen(soup))
            except Exception as e: 
                print('Number '+str(start_num+i)+' Data Encountered Issue:')
                print(e)
                continue
        NP_state = np_check
        start_num +=10
        print('Finished till Paper number '+str(start_num))
        time.sleep(random.randint(4,20))
        
    return ingreds
    

In [397]:
dota2_data = scholar_crawler('dota2',2016, 2017)

Start working on Crawling Job: 「Paper related to dota2 from 2016 to 2017」


HTTPError: 503 Server Error: Service Unavailable for url: https://www.google.com/sorry/index?continue=https://scholar.google.com.tw/scholar%3Fstart%3D0%26q%3Ddota2%26hl%3Den%26as_sdt%3D0%252C9%26as_ylo%3D2017%26as_yhi%3D2016&hl=en&q=EgRo7i9CGJ_bwN4FIhkA8aeDS-_IASyxo9WBTve8vyP8-VUd93pJMgFy

In [401]:
print(requests.utils.default_headers())

{'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [403]:
random.randint(4,20)

8